In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CO_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,8.031004e+13
1,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,8.041004e+13
2,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,8.031007e+13
3,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,8.014031e+13
4,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,8.031004e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,8.031004e+13
1,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,8.041004e+13
2,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,8.031007e+13
3,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,8.014031e+13
4,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,8.031004e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6,8013527003,CO,"14481 HUDSON ST, THORNTON, CO",39.959302,-104.923150,NaN
581,4884168409,CO,"717 E INDUSTRIAL BLVD, PUEBLO, CO",38.333472,-104.710716,NaN
1536,3491228408,CO,"919 West Costilla St, Colorado Springs, CO",38.827923,-104.843132,NaN
1743,5077547004,CO,"726 W MAIN ST, PIERCE, CO",39.978562,-86.138417,NaN
3464,4834677001,CO,"1301 EASTWIND DRIVE, LAFAYETTE, CO",40.103697,-82.918740,NaN
4105,7156437003,CO,"1 MEADOWBROOK CIR, LITTLETON, CO",39.587364,-105.000567,NaN
6021,5341238709,CO,"8231 E Prentice Ave, GREENWOOD VILLAGE, CO",39.621001,-104.892577,NaN
9528,1893038305,CO,"PO BOX 102170, DENVER, CO",39.739236,-104.990251,NaN
10754,1707987200,CO,"2790 S HWY 287/PO BOX 7932, LOVELAND, CO",33.781669,-98.156119,NaN
10850,8181657703,CO,"601 Main St, FRISCO, CO",39.576676,-106.094993,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('08')] #CO
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/17 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

17


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12834/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6,8013527003,CO,"14481 HUDSON ST, THORNTON, CO",39.959302,-104.923150,[080010600022022]
581,4884168409,CO,"717 E INDUSTRIAL BLVD, PUEBLO, CO",38.333472,-104.710716,[081010029232030]
1536,3491228408,CO,"919 West Costilla St, Colorado Springs, CO",38.827923,-104.843132,[080410024022027]
1743,5077547004,CO,"726 W MAIN ST, PIERCE, CO",39.978562,-86.138417,[]
3464,4834677001,CO,"1301 EASTWIND DRIVE, LAFAYETTE, CO",40.103697,-82.918740,[]
4105,7156437003,CO,"1 MEADOWBROOK CIR, LITTLETON, CO",39.587364,-105.000567,[080050056241009]
6021,5341238709,CO,"8231 E Prentice Ave, GREENWOOD VILLAGE, CO",39.621001,-104.892577,[080050068591003]
9528,1893038305,CO,"PO BOX 102170, DENVER, CO",39.739236,-104.990251,[080310020001009]
10754,1707987200,CO,"2790 S HWY 287/PO BOX 7932, LOVELAND, CO",33.781669,-98.156119,[]
10850,8181657703,CO,"601 Main St, FRISCO, CO",39.576676,-106.094993,[081170003012001]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12834/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6,8013527003,CO,"14481 HUDSON ST, THORNTON, CO",39.959302,-104.923150,080010600022022
581,4884168409,CO,"717 E INDUSTRIAL BLVD, PUEBLO, CO",38.333472,-104.710716,081010029232030
1536,3491228408,CO,"919 West Costilla St, Colorado Springs, CO",38.827923,-104.843132,080410024022027
1743,5077547004,CO,"726 W MAIN ST, PIERCE, CO",39.978562,-86.138417,NaN
3464,4834677001,CO,"1301 EASTWIND DRIVE, LAFAYETTE, CO",40.103697,-82.918740,NaN
4105,7156437003,CO,"1 MEADOWBROOK CIR, LITTLETON, CO",39.587364,-105.000567,080050056241009
6021,5341238709,CO,"8231 E Prentice Ave, GREENWOOD VILLAGE, CO",39.621001,-104.892577,080050068591003
9528,1893038305,CO,"PO BOX 102170, DENVER, CO",39.739236,-104.990251,080310020001009
10754,1707987200,CO,"2790 S HWY 287/PO BOX 7932, LOVELAND, CO",33.781669,-98.156119,NaN
10850,8181657703,CO,"601 Main St, FRISCO, CO",39.576676,-106.094993,081170003012001


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1743,5077547004,CO,"726 W MAIN ST, PIERCE, CO",39.978562,-86.138417,NaN
3464,4834677001,CO,"1301 EASTWIND DRIVE, LAFAYETTE, CO",40.103697,-82.918740,NaN
10754,1707987200,CO,"2790 S HWY 287/PO BOX 7932, LOVELAND, CO",33.781669,-98.156119,NaN
14644,3426817710,CO,"2035 E Iron, Salina, CO",38.840280,-97.611424,NaN
16964,9405158304,CO,"530 1st Ave, Hudson, CO",40.741404,-73.974421,NaN
18916,2731857310,CO,"39018 HIGHWAY 60, BAYFIELD, CO",36.390889,-98.319889,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
6,8013527003,080010600022022
581,4884168409,081010029232030
1536,3491228408,080410024022027
1743,5077547004,NaN
3464,4834677001,NaN
4105,7156437003,080050056241009
6021,5341238709,080050068591003
9528,1893038305,080310020001009
10754,1707987200,NaN
10850,8181657703,081170003012001


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CO_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,164294,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,8.031004e+13
1,164295,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,8.041004e+13
2,164296,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,8.031007e+13
3,164297,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,8.014031e+13
4,164298,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,8.031004e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,164294,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,8.031004e+13,NaN
1,164295,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,8.041004e+13,NaN
2,164296,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,8.031007e+13,NaN
3,164297,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,8.014031e+13,NaN
4,164298,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,8.031004e+13,NaN
...,...,...,...,...,...,...,...,...
19663,183957,9550677710,CO,"4885 Sapphire Dr, Colorado Springs, CO",38.902281,-104.744067,8.041005e+13,NaN
19664,183958,9702248804,CO,"1200 S Parker Rd, Denver, CO",39.694965,-104.895223,8.005087e+13,NaN
19665,183959,9749688303,CO,"2465 S Downing St Ste 210, Denver, CO",39.671859,-104.973805,8.031003e+13,NaN
19666,183960,9863338307,CO,"7674 Grandview Ave Ste 210, Arvada, CO",39.798903,-105.082114,8.059010e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,164294,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,8.031004e+13,NaN,80310041112080.0
1,164295,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,8.041004e+13,NaN,80410037081011.0
2,164296,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,8.031007e+13,NaN,80310068122006.0
3,164297,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,8.014031e+13,NaN,80140311041003.0
4,164298,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,8.031004e+13,NaN,80310038011023.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2453537209,CO,"4624 CENTRAL PARK BLVD, DENVER, CO",39.780484,-104.882268,80310041112080.0
1,3001789003,CO,"1460 Garden of the Gods Rd, Colorado Springs, CO",38.898562,-104.854028,80410037081011.0
2,3541737204,CO,"7555 E. HAMPDEN AVE Suite 520, DENVER, CO",39.654651,-104.901975,80310068122006.0
3,4036967403,CO,"385 Interlocken Crescent Suite 400, BROOMFIELD...",39.923464,-105.117353,80140311041003.0
4,4483637106,CO,"252 CLAYTON ST, DENVER, CO",39.720064,-104.955404,80310038011023.0
...,...,...,...,...,...,...
19663,9550677710,CO,"4885 Sapphire Dr, Colorado Springs, CO",38.902281,-104.744067,80410049022016.0
19664,9702248804,CO,"1200 S Parker Rd, Denver, CO",39.694965,-104.895223,80050868011007.0
19665,9749688303,CO,"2465 S Downing St Ste 210, Denver, CO",39.671859,-104.973805,80310030022011.0
19666,9863338307,CO,"7674 Grandview Ave Ste 210, Arvada, CO",39.798903,-105.082114,80590104052011.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1743,5077547004,CO,"726 W MAIN ST, PIERCE, CO",39.978562,-86.138417,NaN
3464,4834677001,CO,"1301 EASTWIND DRIVE, LAFAYETTE, CO",40.103697,-82.918740,NaN
10754,1707987200,CO,"2790 S HWY 287/PO BOX 7932, LOVELAND, CO",33.781669,-98.156119,NaN
14644,3426817710,CO,"2035 E Iron, Salina, CO",38.840280,-97.611424,NaN
16964,9405158304,CO,"530 1st Ave, Hudson, CO",40.741404,-73.974421,NaN
18916,2731857310,CO,"39018 HIGHWAY 60, BAYFIELD, CO",36.390889,-98.319889,NaN


Convert to File

In [19]:
fips_CO = fips_merge_drop

In [20]:
fips_CO.to_csv("../../../data/state_data/geo/geo_fips/23/CO_fips_scraped.csv") 